In [1]:
import tensorflow as tf
import tensorflow.contrib as tfc
import os
import numpy as np


os.environ['CUDA_VISIBLE_DEVICES'] = '1' #使用 GPU 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
def use_gpu_polite(using_rate=0.6):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = using_rate
    return config

## 探讨单双向RNN和单双向LSTM的输出问题  
RNN比较简单，status只有一个，双向也就简单concat一下就好了。  
LSTM相对复杂，因为status包含c和h，h就是ouput的最后一步。  
**由于双向的结构是外层正反向列表，内层c和h列表。所以要拼接比较麻烦。**  
**千万注意，lstm的status的构成是先c，后h**

In [2]:
x_input = tf.placeholder(tf.float32, [None, 3, 2])
input_len = tf.placeholder(tf.int32, [None])
f_size, b_size = 3, 2

# 单向rnn
rnn_o, rnn_s = tf.nn.dynamic_rnn(tfc.rnn.BasicRNNCell(f_size), x_input, dtype=tf.float32,
                                sequence_length=input_len)
# 双向rnn，所以返回的output和status都是tuple,  
# outputs shaope: [batch_size * time_step * dim_f, ... * dim_b] 两个方向可以不同维度
rnn_b_o, rnn_b_s = tf.nn.bidirectional_dynamic_rnn(tfc.rnn.BasicRNNCell(f_size), tfc.rnn.BasicRNNCell(b_size),
                                                   x_input, dtype=tf.float32, sequence_length=input_len)
rnn_b_o_c = tf.concat(rnn_b_o, axis=-1)
rnn_b_s_c = tf.concat(rnn_b_s, axis=-1)


# 单向lstm, status 分为 c 和 h
lstm_o, (lstm_c, lstm_h) = tf.nn.dynamic_rnn(tfc.rnn.LSTMCell(f_size), x_input,
                                             dtype=tf.float32, sequence_length=input_len)
# 双向lstm
lstm_b_o, (lstm_f_s, lstm_b_s) = tf.nn.bidirectional_dynamic_rnn(tfc.rnn.LSTMCell(f_size), tfc.rnn.LSTMCell(b_size),
                                                   x_input, dtype=tf.float32, sequence_length=input_len)
lstm_b_o_c = tf.concat(lstm_b_o, axis=-1)

lstm_f_c, lstm_f_h = lstm_f_s  # 前向的c和h
lstm_b_c, lstm_b_h = lstm_b_s  # 后向的c和h
lstm_h_concate = tf.concat([lstm_f_h, lstm_b_h], axis=-1)  # 拼接正反的h
lstm_b_o_c_0 = tf.reverse(lstm_b_o_c, axis=[1])[:, 0, :]  # 先按照time step翻转，然后取第一步


# 实际运行环节
x_in = np.random.random([2, 3, 2])
len_in = np.array([2, 3])
config = use_gpu_polite()
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    rbs, rboc, rbsc = sess.run([rnn_b_s, rnn_b_o_c, rnn_b_s_c], {x_input: x_in, input_len: len_in})
    lo, lc, lh = sess.run([lstm_o, lstm_c, lstm_h], {x_input: x_in, input_len: len_in})
    lboc, lbhc = sess.run([lstm_b_o_c, lstm_h_concate], {x_input: x_in, input_len: len_in})
    lboc0 = sess.run(lstm_b_o_c_0, {x_input: x_in, input_len: len_in})
    print('\n11111111.输入的x')
    print(x_in)
    print('\n22222222.双向rnn的output和status，比较两个方向和拼接后的结果，status是正加反：')
    print(rboc)
    print(rbsc)
    print(rbs)
    print('-' * 100)
    print('\n33333333.单向LSTM的output和status中的h，c')
    print(lo)
    print(lh)
    print(lc)
    print('\n44444444.双向LSTM的output，正确的h，巧妙的h')
    print(lboc)
    print(lbhc)
    print(lboc0)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead

11111111.输入的x
[[[0.02503985 0.70159322]
  [0.70557438 0.78248669]
  [0.62221984 0.41850192]]

 [[0.68420447 0.98017391]
  [0.88879729 0.51480725]
  [0.48024142 0.20397805]]]

22222222.双向rnn的output和status，比较两个方向和拼接后的结果，status是正加反：
[[[-0.3485087   0.4817568   0.1782444   0.5397934   0.03973075]
  [-0.00605379  0.2663573   0.11327919 -0.02078193 -0.09425943]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.23104002  0.62741125 -0.10117388  0.2911049  -0.29898652]
  [ 0.03270912 -0.03880924 -0.1780076  -0.36942184 -0.30528158]
  [ 0.0140127   0.09617876 -0.32880777 -0.26975223 -0.10410229]]]
[[-0.00605379  0.2663573   0.11327919  0.5397934   0.03973075]
 [ 0.0140127   0.09617876 -0.32880777  0.2911049  -0.29898652]]
(array([[-0.00605379,  0.2663573 ,  0.11327919],
       [ 0.0140127 ,  0.09617876, -0.32880777]], dtype=floa

## 使用text-lstm的时候，我们一般有两个思路：  
### 第一个是使用每个步长的输出，也即output  
### 第二个就是只取最终输出  
1. 单向很简单，也就是status中的h  
2. 双向比较复杂，需要不断解开tuple，最后拼接，变量“lstm_h_concate”就是这么做到  
3. 另一种方法比较简单，首先把output按照time step翻转，然后取第一步，变量“lstm_b_o_c_0”就是这么做的  
**但是，这是不对的，当seq len不是全长，就会得到0。而且它得到的是正向结尾+反向开头**